In [1]:
import ollama
import requests
import json

In [3]:
model_name = "llama3.1"

#### Task 1: Create a Simple Chain for Summarization


**Objective:**

Build a LangChain chain that can summarize a given text.

**Task Description:**

- Create a LLMChain using with a Ollama model.
- Define a prompt template for summarization.
- Run the chain with a sample text and print the summary.
- Add model output streaming.
- Run the chain with a sample text and print the summary.


In [38]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# Load the Ollama model
llm = ChatOllama(model=model_name)

# Define the prompt template
prompt_template = ChatPromptTemplate([
    ("system", "Your task is to summarize a text in one sentence."),
    ("human", "{text}"),
])

# Create the LLMChain
chain = prompt_template | llm 

# Run the chain with a sample text
text = """Over the last decade, deep learning has evolved massively to process and generate unstructured data like text, images, and video. 
These advanced AI models have gained popularity in various industries, and include large language models (LLMs). 
There is currently a significant level of fanfare in both the media and the industry surrounding AI,
and there’s a fair case to be made that Artificial Intelligence (AI), with these advancements,
is about to have a wide-ranging and major impact on businesses, societies, and individuals alike.
This is driven by numerous factors, including advancements in technology, high-profile applications, 
and the potential for transfor- mative impacts across multiple sectors."""

summary = chain.invoke(text)
print(summary.content)

Deep learning has advanced significantly over the last decade, with large language models and other AI technologies poised to have a major impact on businesses, societies, and individuals due to rapid technological advancements and various transformative potential applications.


In [40]:
# Stream the chain output
for chunk in chain.stream({"text": text}):
    print(chunk.content, end="", flush=True)#

Advances in deep learning are poised to have a significant impact on various industries, businesses, and society as a whole, driven by technological advancements and high-profile applications of AI technologies like large language models.

#### Task 2: Chain with Tool Usage (Simple Math Tool)

**Objective:**

Create a LangChain chain that uses a simple math tool to perform calculations.

**Task Description:**

- Define a function as tool which multiplies two integer values and return the result.
- Create a chain
- Print the result of the calculation.

In [53]:
from langchain_core.tools import tool

# Create custom tool
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

print(multiply.name)
print(multiply.description)
print(multiply.args) # -> definition of tool arguments

# Invoke custom tool
multiply.invoke({"first_int": 4, "second_int": 5})

multiply
Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


20

In [57]:
from langchain_ollama.chat_models import ChatOllama

# Load the Ollama model
llm = ChatOllama(model=model_name)

# Use bind_tools to pass the definition of our tool in as part of each call to the model, so that the model can invoke the tool
llm_with_tools = llm.bind_tools([multiply])

# When the model invokes the tool, this will show up in the AIMessage.tool_calls attribute of the output -> extract tool parameters from input text
msg = llm_with_tools.invoke("whats 5 times forty two")
print(msg.tool_calls)

[{'name': 'multiply', 'args': {'first_int': 5, 'second_int': 42}, 'id': 'aa2a27cb-5b1d-4b14-800e-c5db993a76b0', 'type': 'tool_call'}]


In [58]:
# Create the chain: pass the extracte tool parameters from the input text to the tool -> extract the arguments of the first tool_call
chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply

# Run chain
chain.invoke("What's four times 23")

92

#### Task 3: Agent with Tool Usage (Two Tools)

**Objective:** 

Create a LangChain agent that uses two tools (e.g., a math tool and a customer service tool) to perform tasks.

**Task Description:**

- Define prompt template.
- Define tools.
- Create an Agent using the Ollama model, prompt template and tools.
- Run the agent with a prompt that requires one or both tools.
- Observe how the agent uses the tools to complete the task.

#### Task 4: Enhance Agent with Memory

**Objective:**

Eenhance the agent from Task 3 with memory to improve its context awareness and ability to maintain state.

**Instructions:**

- Create a ConversationBufferMemory to store chat history.
- Modify the agent to use the memory to inform its responses.
- Run the agent with a series of prompts that require context or state to be maintained.
- Observe how the agent's responses improve with the addition of memory.